In [ ]:
%matplotlib inline

# system modules
import os
import time
from pathlib import Path
from IPython.display import display, Math

# scientific computing
import numpy as np
from numpy import linalg as LA
import pandas as pd
np.random.seed(42)
from fipy import CellVariable,PeriodicGrid2D
from fipy import DiffusionTerm, ExponentialConvectionTerm, DefaultAsymmetricSolver, ImplicitSourceTerm
from fipy import MatplotlibStreamViewer
from fipy.tools.numerix import array, reshape

# plotting
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import plotly.io as pio
from plotly import subplots
init_notebook_mode(connected=True)

# pytorch importing
import torch
import torch.nn as nn
from torchvision import datasets
from torch.optim import lr_scheduler, Adam
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
torch.manual_seed(42)

### define the calculation domain

In [ ]:
# source field = calculation domain
dx = 0.01; dy = 0.01
Nx = 100; Ny = 100
Lx = dx*Nx; Ly = dy*Ny
mesh2 = PeriodicGrid2D(dx=dx, dy=dy, nx=298, ny=298)

In [ ]:
mesh2.shape

In [ ]:
epsilon=2e-1 # error tolerance in defining the influence region
n=6
cells=np.array([[-5,-4,-3,-2,-1,0,1,2,3,4,5]])
cells_x=np.repeat(cells,cells.shape[1],axis=0)
cells_y=np.repeat(cells.T,cells.shape[1],axis=1)

### calculate the periodic covariance matrix

In [ ]:
# # hyperparameters
# sigma = 1; l1 = 0.1; l2 = 0.1

# # source mesh points
# src_xy = mesh.cellCenters.value

# # rbf_kernel (create a specified covariance matrix)
# '''
#         rbf_kernel:  sigma**2 exp(-(|x_i - x_j|)**2 / (2*l**2))
# '''
# cov = np.zeros([Nx*Ny,Nx*Ny])
# for j in range(Nx*Ny):
#     for i in range(Nx*Ny):
#         cov[i,j] = sigma**2 * np.exp(-((src_xy[0,i]-src_xy[0,j])**2 / (2*l1**2) + (src_xy[1,i]-src_xy[1,j])**2 / (2*l2**2)))

# # Make sure covariance matrix is symmetric
# a = np.allclose(cov, cov.T, rtol=1e-05, atol=1e-08)

# cov = cov + 1e-10*np.eye(np.shape(cov)[0])
# # Make sure your matrix is positve definite
# b = np.any(np.linalg.eigvalsh(cov) < 0.)

# if a==False:
#     print('ERROR: Covariance matrix is not symmetric as required')
# elif b==True:
#     print('ERROR: Covariance matrix is not positive definite as required')
#     print('eigenvalues: ', np.linalg.eigvalsh(cov))

# # Decompose covariance matrix (Cholesky Decomposition)
# L = np.linalg.cholesky(cov)
# Lt = L.T
# L_Lt = np.dot(L,Lt)

# a = np.allclose(cov, L_Lt, rtol=1e-05, atol=1e-08)
# if a==True:
#     print('Successfully decomposed!')
# else:
#     print('ERROR: L_Lt != cov')

In [ ]:
# np.save('cov_matrix.npy',L)

In [ ]:
L = np.load('cov_matrix_periodic.npy')

### Random source field

In [ ]:
def get_source_field():
    u = np.random.normal(size=(Nx*Ny,1))    # generate uncorrelated standard normal
    samples = np.dot(L,u)
    samples = (5/(samples.max()-samples.min()))*(samples-samples.min())+0
    samples_final = samples.reshape(Ny,Nx)
    samples_final[-1,:] = samples_final[0,:]
    samples_final[:,-1] = samples_final[:,0]   # make sure the source is indeed periodic!
    return samples_final

In [ ]:
def peri_source(source):
    a = np.concatenate((source[0:-1,:],source,source[1:,:]))
    b = np.concatenate((a[:,0:-1],a,a[:,1:]),axis=1)
    return b

### Source Function

In [ ]:
# def sample_src(mesh1,source):
#     src = CellVariable(name='Source', mesh=mesh1)   
# #     u = np.random.normal(size=(Nx*Ny,1))    # generate uncorrelated standard normal
# #     samples = np.dot(L,u)
# #     samples_final= (5/(samples.max()-samples.min()))*(samples-samples.min())+0
#     samples_final = source.flatten()
#     src.value = np.ravel(samples_final)     # reshape sample_final (10000,1) to 10000
    
#     return src

In [ ]:
def sample_src2(mesh2,source):
    src = CellVariable(name='Source_big', mesh=mesh2)   
#     u = np.random.normal(size=(Nx*Ny,1))    # generate uncorrelated standard normal
#     samples = np.dot(L,u)
#     samples_final= (5/(samples.max()-samples.min()))*(samples-samples.min())+0
    samples_final = (peri_source(source[::-1])[::-1]).flatten()
    src.value = np.ravel(samples_final)     # reshape sample_final (90000,1) to 90000
    
    return src

### Velocity Function

In [ ]:
def sample_vel(mesh, C_u, alpha):
    vel = mesh.cellCenters.copy()
    vel.name = 'Velocity'
    vel.value[0] = C_u * np.cos(alpha)
    vel.value[1] = C_u * np.sin(alpha)
    return vel

### Data Generation Function

In [ ]:
# def sample_data(mesh, C_uniform, alpha, diff_coeff, diss_coeff):
#     data_list = []
#     array_list = []
#     var = CellVariable(name='Variable',mesh=mesh)
#     src = sample_src(mesh,random_source)
#     vel = sample_vel(mesh, C_uniform, alpha)
#     eq = - ExponentialConvectionTerm(coeff=vel) + DiffusionTerm(coeff=diff_coeff) - ImplicitSourceTerm(diss_coeff) + src
#     eq.solve(var=var, solver=DefaultAsymmetricSolver(tolerance=1.e-12, iterations=10000))
#     data = {'var': var, 'src': src, 'vel': vel, 'diff': diff_coeff, 'diss': diss_coeff}
#     return data

In [ ]:
def sample_data_big(mesh, C_uniform, alpha, diff_coeff, diss_coeff):
    data_list = []
    array_list = []
    var = CellVariable(name='Variable',mesh=mesh)
    src = sample_src2(mesh,random_source)
    vel = sample_vel(mesh, C_uniform, alpha)
    eq = - ExponentialConvectionTerm(coeff=vel) + DiffusionTerm(coeff=diff_coeff) - ImplicitSourceTerm(diss_coeff) + src
    eq.solve(var=var, solver=DefaultAsymmetricSolver(tolerance=1.e-12, iterations=10000))
    data = {'var': var, 'src': src, 'vel': vel, 'diff': diff_coeff, 'diss': diss_coeff}
    return data

### Data Storage

In [ ]:
# def periodic(array,h):  # dimension of array => [Ny,Nx]
#     peri_num_x = 2*(int(N*h/Lx)+1)+1
#     peri_num_y = 2*(int(N*h/Ly)+1)+1
#     peri = np.zeros((peri_num_y*Ny,peri_num_x*Nx))
#     for i in range(0,Ny*(peri_num_y-1)+1,Ny):
#         for j in range(0,Nx*(peri_num_x-1)+1,Nx):
#             peri[i:i+Ny,j:j+Nx] = array
#     return peri

### visualization the field

In [ ]:
def view_cell(var):
    fig, axes = plt.subplots(1, 1, figsize=(4, 2.4))
    axes.set_title('{}'.format(var.name))
    axes.set_xlabel('x')
    axes.set_ylabel('y')
    cmap = matplotlib.cm.viridis
    xmin, ymin = var.mesh.extents['min']
    xmax, ymax = var.mesh.extents['max']
    data = reshape(array(var), var.mesh.shape[::-1])[::-1]
    img = axes.imshow(data, extent=(xmin, xmax, ymin, ymax), cmap=cmap.reversed())
    plt.colorbar(img)

## Training and testing data from different uniform flow cases

### Data Generation - testing

In [ ]:
num_samples_valid=5

for sample in range(num_samples_valid):
    # PDE coefficients
    random_source = get_source_field()
    
    diff_coeff = 0.05
    diss_coeff = 12

    # flow parameters
    C_max = 2.0; C_min = 0.1 
    C_uniform = np.random.uniform(low=C_min, high=C_max)     # Uniform Flow Strength
    alpha = 0 
    
    data_3_valid = sample_data_big(mesh2, C_uniform, alpha, diff_coeff, diss_coeff)

    lambda1=(C_max - (((C_max**2)+(4*diff_coeff*diss_coeff))**0.5)) / (2*diff_coeff)
    h=np.abs(np.log(epsilon)/lambda1)   # h => maximum h

    print('diff     diss     U     h')
    print('{:.3f}   {:.3f}   {:.3f}  {:.3f}'.format(diff_coeff,diss_coeff,C_uniform,h))
    view_cell(data_3_valid['src'])
    view_cell(data_3_valid['var'])

    c_data_valid_mesh_3 = reshape(array(data_3_valid['var']), data_3_valid['var'].mesh.shape)[::-1]
    s_data_valid_mesh_3 = reshape(array(data_3_valid['src']), data_3_valid['src'].mesh.shape)[::-1]
    u_data_valid_mesh_3 = reshape(array(data_3_valid['vel'])[0,:], data_3_valid['vel'].mesh.shape)[::-1]
    v_data_valid_mesh_3 = reshape(array(data_3_valid['vel'])[1,:], data_3_valid['vel'].mesh.shape)[::-1]

    dataX_valid_current=np.empty([10000, 3, cells.shape[1], cells.shape[1]])
    dataY_valid_current=np.empty([10000, 1])

    data_ind=0
    for i in range(99,199):
        for j in range(99,199):
            dataX_valid_current[data_ind,0,:,:]=u_data_valid_mesh_3[(cells_y*n)+i,(cells_x*n)+j]
            dataX_valid_current[data_ind,1,:,:]=v_data_valid_mesh_3[(cells_y*n)+i,(cells_x*n)+j]
            dataX_valid_current[data_ind,2,:,:]=s_data_valid_mesh_3[(cells_y*n)+i,(cells_x*n)+j]
            dataY_valid_current[data_ind,0]=c_data_valid_mesh_3[i,j]
            
            data_ind+=1

    dataX_valid_current=torch.tensor(dataX_valid_current).to(dtype=torch.float)
    dataY_valid_current=torch.tensor(dataY_valid_current).to(dtype=torch.float)

    if sample<1:
        dataX_valid=dataX_valid_current
        dataY_valid=dataY_valid_current
        
    else:
        dataX_valid=torch.cat((dataX_valid,dataX_valid_current))
        dataY_valid=torch.cat((dataY_valid,dataY_valid_current))
        

tensor_valid = {'X': dataX_valid, 'Y': dataY_valid}
torch.save(tensor_valid, 'valid_11_%d'%(n)) 

### Data-Generation - Training

In [ ]:
num_samples=100

for sample in range(num_samples):
    
    random_source = get_source_field()
    
    # PDE coefficients
    diff_coeff = 0.05
    diss_coeff = 12

    # flow parameters
    C_max = 2.0; C_min = 0.1
    C_uniform = np.random.uniform(low=C_min, high=C_max)     # Uniform Flow Strength
    alpha = 0 

    data_3 = sample_data_big(mesh2, C_uniform, alpha, diff_coeff, diss_coeff)

    lambda1=(C_max - (((C_max**2)+(4*diff_coeff*diss_coeff))**0.5)) / (2*diff_coeff)
    h=np.abs(np.log(epsilon)/lambda1)   # h => maximum h

    c_data_mesh_3 = reshape(array(data_3['var']), data_3['var'].mesh.shape)[::-1]
    s_data_mesh_3 = reshape(array(data_3['src']), data_3['src'].mesh.shape)[::-1]
    u_data_mesh_3 = reshape(array(data_3['vel'])[0,:], data_3['vel'].mesh.shape)[::-1]
    v_data_mesh_3 = reshape(array(data_3['vel'])[1,:], data_3['vel'].mesh.shape)[::-1]

    dataX_train_current=np.empty([10000, 3, cells.shape[1], cells.shape[1]])
    dataY_train_current=np.empty([10000, 1])
    
    data_ind=0
    for i in range(99,199):
        for j in range(99,199):
            dataX_train_current[data_ind,0,:,:]=u_data_mesh_3[(cells_y*n)+i,(cells_x*n)+j]
            dataX_train_current[data_ind,1,:,:]=v_data_mesh_3[(cells_y*n)+i,(cells_x*n)+j]
            dataX_train_current[data_ind,2,:,:]=s_data_mesh_3[(cells_y*n)+i,(cells_x*n)+j]
            dataY_train_current[data_ind,0]=c_data_mesh_3[i,j]
             
            data_ind+=1

    dataX_train_current=torch.tensor(dataX_train_current).to(dtype=torch.float)
    dataY_train_current=torch.tensor(dataY_train_current).to(dtype=torch.float)

    if sample<1:
        dataX_train=dataX_train_current
        dataY_train=dataY_train_current
    else:
        dataX_train=torch.cat((dataX_train,dataX_train_current))
        dataY_train=torch.cat((dataY_train,dataY_train_current))

tensor_train = {'X': dataX_train, 'Y': dataY_train}
torch.save(tensor_train, 'train_11_%d'%(n))

### Neural network architecture

In [ ]:
class CNN_Network(nn.Module):
    def __init__(self):
        super(CNN_Network, self).__init__()
        
        self.relu = nn.ReLU()
        
        self.conv1 = nn.Conv2d(in_channels=2, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=8, out_channels=4, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(in_channels=4, out_channels=1, kernel_size=3, stride=1, padding=1)
        
        
    def forward(self, X):
        X1 = X[:,0:2,:,:]
        X2 = X[:,2:3,:,:]
        out = self.relu(self.conv1(X1))
        out = self.relu(self.conv2(out))
        out = self.relu(self.conv3(out))
        out = self.relu(self.conv4(out))
        out = self.relu(self.conv5(out))
        G = self.conv6(out)
        dim1 = X.size()[0]
        c = torch.sum(G*X2,axis=(1,2,3))
        c = c.reshape(dim1,1)
        
        return G,c

### Training Function

In [ ]:
def train(train_loader, valid_loader, num_epoch):
    train_err_hist = torch.cuda.FloatTensor(1,1).fill_(0)
    valid_err_hist = torch.cuda.FloatTensor(1,1).fill_(0)
    train_loss_hist = torch.cuda.FloatTensor(1,1).fill_(0)
    valid_loss_hist = torch.cuda.FloatTensor(1,1).fill_(0)

    for epoch in range(num_epoch+1):
        train_loss_array = torch.cuda.FloatTensor(1,1).fill_(0)
        train_err_rate_num = torch.cuda.FloatTensor(1,1).fill_(0)
        train_err_rate_den = torch.cuda.FloatTensor(1,1).fill_(0)
        valid_loss_array = torch.cuda.FloatTensor(1,1).fill_(0)
        valid_err_rate_num = torch.cuda.FloatTensor(1,1).fill_(0)
        valid_err_rate_den = torch.cuda.FloatTensor(1,1).fill_(0)
        

        for i, data in enumerate(train_loader):
            features, target = data
            optimizer.zero_grad()
            G_train, forward = model(features)
            loss = loss_fn(forward, target)
            loss.backward()
            optimizer.step()

            train_loss_array = torch.cat((train_loss_array, torch.cuda.FloatTensor([[loss.item()]])))
            train_err_num, train_err_den = report_err_rate(target, forward)
            train_err_rate_num = torch.cat((train_err_rate_num, (train_err_num.view(1,-1))**2), 0)
            train_err_rate_den = torch.cat((train_err_rate_den, (train_err_den.view(1,-1))**2), 0)

        train_loss = torch.mean(train_loss_array)
        train_err_rate = 100*((torch.sum(train_err_rate_num, 0))**0.5)/((torch.sum(train_err_rate_den, 0))**0.5)

        exp_lr_scheduler.step()

        with torch.no_grad():
            for i, data_valid in enumerate(valid_loader):
                features_valid, target_valid = data_valid
                G_valid, forward_valid = model(features_valid)
                pred_loss = loss_fn(forward_valid, target_valid)

                valid_loss_array = torch.cat((valid_loss_array, torch.cuda.FloatTensor([[loss.item()]])))
                valid_err_num, valid_err_den = report_err_rate(target_valid, forward_valid)
                valid_err_rate_num = torch.cat((valid_err_rate_num, (valid_err_num.view(1,-1))**2), 0)
                valid_err_rate_den = torch.cat((valid_err_rate_den, (valid_err_den.view(1,-1))**2), 0)

            valid_loss = torch.mean(valid_loss_array)
            valid_err_rate = 100*((torch.sum(valid_err_rate_num, 0))**0.5)/((torch.sum(valid_err_rate_den, 0))**0.5)
        
        if ((train_err_rate <= 2.5) and (valid_err_rate <= 2.5)):
            torch.save(model, 'model_gpu.pt')
        
        verb = True if (epoch >= 50) and (epoch % 10 == 0) else False
        if (verb):
            train_loss_hist = torch.cat((train_loss_hist, torch.cuda.FloatTensor([[train_loss]])))
            train_err_hist = torch.cat((train_err_hist, train_err_rate.view(1,-1)), 0)
            valid_loss_hist = torch.cat((valid_loss_hist, torch.cuda.FloatTensor([[valid_loss]])))
            valid_err_hist = torch.cat((valid_err_hist, valid_err_rate.view(1,-1)), 0)
        verb = True if (epoch % 50 == 0) else False
        if (verb) :
            print('{:4}   lr: {:.2e}   train_loss: {:.2e}   valid_loss: {:.2e}   train_error:{:7.2f}%   valid_error:{:7.2f}%' \
                  .format(epoch, exp_lr_scheduler.get_lr()[0], train_loss, valid_loss, train_err_rate[0], valid_err_rate[0]))
            
    print('Finished Training')
    return train_loss_hist, train_err_hist, valid_loss_hist, valid_err_hist

In [ ]:
def report_err_rate(target, forward):
    errRate_sigma_num = torch.norm(forward - target, dim = 0)
    errRate_sigma_den = torch.norm(target, dim = 0)
    return errRate_sigma_num, errRate_sigma_den

### data load

In [ ]:
tensor_valid = torch.load('valid_11_6')
tensor_train = torch.load('train_11_6')

In [ ]:
dataX_valid = tensor_valid['X']
dataY_valid = tensor_valid['Y']
dataX_train = tensor_train['X']
dataY_train = tensor_train['Y']

In [ ]:
reduc_data_size = int(dataX_train.shape[0] * (50/100)) # downsampling
ind = list(range(dataX_train.shape[0]))
np.random.shuffle(ind)
train_ind = ind[:reduc_data_size]
dataX_train = dataX_train[train_ind]
dataY_train = dataY_train[train_ind]
print('Reduced Training Data Size: {}   {}'.format(dataX_train.shape, dataY_train.shape))
print('Validation Data Size:       {}   {}'.format(dataX_valid.shape, dataY_valid.shape))

In [ ]:
if torch.cuda.is_available():
    device=torch.device('cuda:0')
else:
    device=torch.device('cpu')
    
device_cpu = torch.device('cpu')

In [ ]:
dataX_train = dataX_train.to(device)
dataY_train = dataY_train.to(device)
dataX_valid = dataX_valid.to(device)
dataY_valid = dataY_valid.to(device)

In [ ]:
#creating datasets
dataset_train = TensorDataset(dataX_train,dataY_train)
dataset_valid = TensorDataset(dataX_valid,dataY_valid)

#creating batches from dataset
batch_size_train = 1024       
batch_size_valid = dataX_valid.shape[0]

train_loader = DataLoader(dataset = dataset_train, batch_size=batch_size_train, shuffle=True)
valid_loader = DataLoader(dataset = dataset_valid, batch_size=batch_size_valid, shuffle=False)

In [ ]:
np.random.seed(7)
model = CNN_Network()
model.to(device)
loss_fn = nn.MSELoss(reduction='sum')

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
para_count = count_parameters(model)
print('Total Learnable Parameters: {}'.format(para_count))

In [ ]:
# training
num_epoch = 2000
learning_rate = 1e-3
optimizer = Adam(model.parameters(), lr=learning_rate)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=600, gamma=0.7)

In [ ]:
start_time = time.time()
training_loss_history, training_error_history, valid_loss_history, valid_error_history = train(train_loader, valid_loader, num_epoch)
elapsed = time.time() - start_time                
print('Training time: %.1f s' % (elapsed))

In [ ]:
torch.save(model, 'model_gpu_final.pt')
model.to(device_cpu)
torch.save(model, 'model_cpu_final.pt')